In [2]:
from clef.utils.data_loading import load_datasets
from clef.verification.verify import check_dataset_with_model
from clef.utils.data_loading import write_jsonlines_from_dicts
import os

root_path = '../'

train, dev = load_datasets(preprocess=True,
                           add_author_name=True,
                           add_author_bio=False,
                           root_path= root_path,)


loaded 96 training json lines and 32 dev json lines.


In [2]:
import json
print(json.dumps(dev[0], indent=2))

{
  "id": "AuRED_142",
  "rumor": "Naturalization decree in preparation: Lebanese passports for sale !",
  "label": "REFUTES",
  "timeline": [
    [
      "https://twitter.com/LBpresidency",
      "1556600039211925504",
      "Account: Lebanese Presidency\nText: Today the President of the Republic General Michel Aoun signed 9 laws that were previously approved by the House of Representatives Details at the following link:"
    ],
    [
      "https://twitter.com/LBpresidency",
      "1556559119045332992",
      "Account: Lebanese Presidency\nText: President Aoun received the Minister of Foreign Affairs and Expatriates Dr Abdullah Bouhabib and the Minister of Social Affairs Hector Hajjar and discussed with them developments related to the file of displaced Syrians in Lebanon"
    ],
    [
      "https://twitter.com/LBpresidency",
      "1556558220533157890",
      "Account: Lebanese Presidency\nText: President Aoun received the new British Ambassador to Lebanon Hamish Cowell and discuss

In [3]:
from clef.verification.verify import factcheck_using_evidence
from clef.verification.models.openai import get_completion
import json

spot = dev[0]
rumor = spot["rumor"]
evidence = spot["evidence"]

def inference_openai(statement: str, evidence: str):
    input_text = f'The statement: "{evidence}"\nThe claim: "{statement}"'

    valid_labels = [
        "REFUTES",
        "NOT ENOUGH INFO",
        "SUPPORTS"
    ]
    
    result = get_completion(input_text).content

    try:
        decision, confidence = json.loads(result).values()
    except ValueError:
        print(f'ERROR: could not parse response from openai model: {result}')
        return ("NOT ENOUGH INFO", 1.0)

    if decision in valid_labels:
        return (decision, confidence)
    else:
        return ("NOT ENOUGH INFO", 1.0)

print(rumor)
for ev in evidence:
    print(ev[2])
    print(inference_openai(rumor, ev[2]))

Naturalization decree in preparation: Lebanese passports for sale !
Account: Lebanese Presidency
Text: The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine Liberation about the same fabricated news
('REFUTES', 0.95)
Account: Lebanese Presidency
Text: The Information Office of the Presidency of the Republic: What was published by the French newspaper Liberation about the selling of Lebanese passports to non-Lebanese is false and baseless news
('REFUTES', 0.95)


In [10]:
out = '{"decision": "REFUTES", "confidence": 0.95}'
decision, score = json.loads(out).values()
score

0.95

In [11]:
decision

'REFUTES'

sanity checks

In [4]:
claim = 'i saw the sky is a shade of rose today!!'
premise = 'Account: Ministry of Sky color\nText: The sky is pink, and always was.'

inference_openai(claim, premise)

('SUPPORTS', 0.8)

In [5]:
claim = 'i saw the sky is a shade of green today!!'
premise = 'Account: Ministry of Sky color\nText: The sky is pink, and always was.'

inference_openai(claim, premise)

('REFUTES', 0.95)

In [6]:
claim = 'i saw the firmament is a shade of green today!!'
premise = 'Account: Ministry of Sky color\nText: The sky is pink, and always was.'

inference_openai(claim, premise)

('REFUTES', 0.95)